In [3]:
import numpy as np
import pandas as pd
import os
import torch
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import torch.nn as nn
import torch.nn.functional as F
import albumentations as A
import torch.optim as optim 
from torch_lr_finder import *
from torch_lr_finder import LRFinder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy import asarray
from fastai import *
from fastai.vision import *
from sklearn.model_selection import StratifiedKFold
import glob

In [ ]:
if not os.path.exists('checkpoints/'):
        os.makedirs('checkpoints/')


In [ ]:
train_tfm = A.Compose([
     
                A.ShiftScaleRotate(shift_limit = 0.1,
                                    scale_limit = 0.1,
                                    rotate_limit = 30),
                A.Normalize(mean=[0.1307], std=[0.3081])

            ])

      

def valid_tfm(size):
    return A.Compose([ A.Resize(size, size) ])

tfms = transforms.Compose([
    
    transforms.RandomRotation(degrees=(-10, 10)),
    transforms.RandomAffine(degrees=(-16, 16)),
    #transforms.Normalize(mean=(0.5,), std=(0.5,))
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
                           
                          ])
v_tfms = transforms.Compose([
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

In [ ]:
label = []
path = []
label_conversion = {'01': 'neutral',
                            '02': 'happy',
                            '03': 'sad',
                            '04': 'angry',
                            '05': 'fear',
                            '06': 'disgust'}
        
    
for subdirs, dir_, files in os.walk('specs_train'):
    for filenames in files:
        for emotion_number, emotions in label_conversion.items():
            if(filenames[6:8] == emotion_number):
                path.append(os.path.join(subdirs, filenames))
                label.append(emotions)
df = pd.DataFrame(label, columns = ['label'])
df = pd.concat([df,pd.DataFrame(path, columns = ['path'])],axis=1)
df.label.value_counts()
df.to_csv('final_specs.csv', index = False)
        

In [ ]:
final_df  = pd.read_csv('final_specs.csv')
final_df.label.value_counts()

In [ ]:
X, y = final_df['path'].values, final_df['label'].values

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=0.2, random_state = 42)

In [4]:
np.random.seed(42)
data = ImageDataBunch.from_folder(specs_train, train='.', valid_pct=0.2,
        ds_tfms=None, size=[224,224], num_workers=4).normalize(imagenet_stats)

NameError: name 'ImageDataBunch' is not defined